In [ ]:
#Imports
import numpy as np
import tensorflow as tf
from othello import Board
from copy import deepcopy
import time

#Define print de matrizes inteiras no numpy
np.set_printoptions(threshold=np.nan)

class Statistics:
    def __init__(self, tensorboard_file):
        self.tensorboard_file = tensorboard_file
    
    #Salva dados de variaveis para o tensorboard 
    def add_data(self, data):
        data = data if type(data) == list or type(data) == np.ndarray else [data]
        for var in data:
            #Calcula media
            mean = tf.reduce_mean(var)
            #Salva para tensorboard
            tf.summary.scalar('mean', mean)        
            #Calcula desvio padrão
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            #Salva desvio padrão para tensorboard
            tf.summary.scalar('stddev', stddev)
            #Salva maximo para tensorboard
            tf.summary.scalar('max', tf.reduce_max(var))
            #Salva minimo para tensorboard
            tf.summary.scalar('min', tf.reduce_min(var))
            #Salva histogram para tensorboard
            tf.summary.histogram('histogram', var)
            
    def start(self):
        #Inicializa e configura tensorBoard
        self.merged_summary = tf.summary.merge_all()
        self.writer = tf.summary.FileWriter(self.tensorboard_file)

class Game:    
    def __init__(self):
        self.board = Board()
        self.BLACK = 1
        self.WHITE = -1
           
    #Efetua a proxima action com o agente
    def player_move(self, brain, player):
        #Define o board de acordo com o player da vez
        temp_board = (self.board.board * player).reshape([1,-1])

        #Recebe do board todos os movimentos permitidos ao player atual
        moves = self.board.possible_moves(player) 
        
        #Se não existir nenhuma ação que possa ser tomada
        if(len(moves) == 0):
            #retorna avisando que o player não pode se mover
            return -1
        
        #Calcula probabilidades baseadas nas possiveis rewards futuras para definir movimento a ser tomado
        probs = brain.inference(temp_board)

        #Calcula rewards para todos os movimentos
        new_probs = np.zeros(brain.INPUT_DIM)
        
        #Monta lista de probabilidades para todos os movimentos possiveis
        for x, y, _ in moves:
            new_probs[x*8 + y] = probs[x*8 + y]

        #Se a rede achar que nao deve tomar nenhuma ação
        if(np.sum(new_probs) == 0):
            #Sorteia um movimento a ser tomado
            selected_move = np.random.choice(len(moves), 1, p = [1/len(moves)]*len(moves))[0]
            selected_move = moves[selected_move][0]*8 + moves[selected_move][1]
        else: #Caso existam movimentos com probabilidades de serem tomadas
            #Sorteia um movimento baseado nas probabilidades
            new_probs = new_probs/np.sum(new_probs)
            selected_move = np.random.choice(brain.INPUT_DIM, 1, p=new_probs)[0]

        #Efetua movimento no board
        self.board.move(selected_move//8, selected_move%8, player)
        #retona movimento tomado
        return selected_move
    
    
    #Simula um jogo inteiro e retorna (Boards, actions, rewards) de todo o jogo para ambos os players (Rede X Rede)
    def play(self, brain):
        #Inicia um novo board
        self.board.reset()

        #Define listas de States e Actions
        board_history_p1 = []
        board_history_p2 = []
        move_sequence_p1 = []
        move_sequence_p2 = []

        #Printa estado inicial se verbose = true
        brain.v or print(self.board)
        #Toma jogadas ate que o jogo termine
        while(not self.board.finished()):
            #Guarda estado anterior
            board_before_move_p1 = deepcopy(self.board.board)
            #Faz melhor movimento
            p1_move = self.player_move(brain, self.BLACK)
            #Se o player pode se mover
            if(p1_move != -1):
                #Registra estado
                board_history_p1.append(board_before_move_p1)
                #Registra action
                move_sequence_p1.append(p1_move)

            #Guarda estado anterior
            board_before_move_p2 = deepcopy(self.board.inverted_board())
            #Faz melhor movimento
            p2_move = self.player_move(brain, self.WHITE)
            #Se o player pode se movere
            if(p2_move != -1):
                #Registra estado
                board_history_p2.append(board_before_move_p2)
                #Registra action
                move_sequence_p2.append(p2_move)
                
            #Printa round status se verbosee = true
            brain.v or print("====================")
            brain.v or print((move_sequence_p1[-1]//8, move_sequence_p1[-1]%8))
            brain.v or print(self.board)
            brain.v or print("====================")
            brain.v or print((move_sequence_p2[-1]//8, move_sequence_p2[-1]%8))
            brain.v or print(self.board)
        brain.v or print(self.board.score())

        #Calcula reward para todas as ações de BLACK
        reward_sequence_p1 = brain.calculate_reward(self.BLACK, move_sequence_p1)
        #Calcula reward para todas as ações de WHITE
        reward_sequence_p2 = brain.calculate_reward(self.WHITE, move_sequence_p2)

        #Concatena todos os estados de BLACK e WHITE
        board_history = board_history_p1 + board_history_p2
        #Concatena todas as actions de BLACK e WHITE
        move_sequence = move_sequence_p1 + move_sequence_p2
        #Concatena todas as rewards de BLACK e WHITE
        reward_sequence = reward_sequence_p1 + reward_sequence_p2

        #retorna estados, actions e rewards do jogo atual
        return (board_history, move_sequence, reward_sequence) 
    
    #Toma uma jogada aleatoria
    def random_move(self, player):
        #Recebe movimentos validos do board
        moves = self.board.possible_moves(player) 

        #Se não existir um movimento possivel retorna -1
        if(len(moves) == 0):
            return -1

        #Seleciona aleatoriamente um movimento
        selected_move = np.random.choice(len(moves), 1, p = [1/len(moves)]*len(moves))[0]
        #Efetua movimento no tabuleiro
        self.board.move(moves[selected_move][0], moves[selected_move][1], player)         
        
        
class Brain:
    def __init__(self, data_type = tf.float32, restore_file = "", tensorboard_file = './data', learning_rate = 0.01, batch_size = 64, epochCount = 100000, replay_memory_limit = 10000, gamma = 0.95, h_layers_dim = [128, 256, 128], eval_test_size = 100, gpu_num = 1, dynamic_allocation = True, verbose = False):
        self.reset_graph()
        self.data_type = data_type
        self.restore_file = restore_file
        self.batch_size = batch_size
        self.epoch_count = epochCount
        self.replay_memory_limit = replay_memory_limit
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.INPUT_DIM = 8*8
        self.statistics = Statistics(tensorboard_file)
        self.graph = self.build_graph(h_layers_dim)
        self.replay_memory = {
            "states": [],
            "actions": [],
            "rewards": []
        }
        self.v = not verbose

        config = tf.ConfigProto()
        config.gpu_options.allow_growth = dynamic_allocation
        self.gpu_num = gpu_num #UNUSED
                    
        self.sess = tf.Session(config=config)
        
        #Instancia um Saver usado para salvar e recuperar os pesos
        self.saver = tf.train.Saver()
        
        self.game = Game()
        
        self.eval_test_size = eval_test_size


    #Reseta computational graph
    def reset_graph(self):
        #Verifica se existe uma sessão aberta
        if("sess" in globals() and sess):
            #Fecha a sessão
            self.sess.close()
        #Destroi o grafo remanescente
        tf.reset_default_graph()
    
    
    #Constroi computational graph
    def build_graph(self, h_layers_dim):
        with tf.variable_scope("Place_holders"):
            #Cria placeholder para receber o Estado Atual [batch, board]
            input_states = tf.placeholder(self.data_type, [None, self.INPUT_DIM])
            #Cria placeholder para receber as actions [actionNum]
            actions = tf.placeholder(tf.int32, [None])
            #Cria placeholder para receber as rewards [RewardVal]
            rewards = tf.placeholder(self.data_type, [None])

        #Array de Pesos
        weights = []
        #Array de Biases
        biases = []

        with tf.variable_scope("Weights"):
            #Cria Primeira camada (camada q recebe input)
            weights.append(tf.get_variable("W0", [self.INPUT_DIM, h_layers_dim[0]], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))
            #Cria Segunda camada (camada q recebe dados da camada[0])
            weights.append(tf.get_variable("W1", [h_layers_dim[0], h_layers_dim[1]], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))
            #Cria Terceira camada (camada q recebe dados da camada[1])
            weights.append(tf.get_variable("W2", [h_layers_dim[1], h_layers_dim[2]], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))
            #Cria Quarta camada (camada de output) (camada q recebe dados da camada[2])
            weights.append(tf.get_variable("W3", [h_layers_dim[2], self.INPUT_DIM], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))

            #Calcula dados e adiciona ao tensorboard
            self.statistics.add_data(weights)

        with tf.variable_scope("Biases"):
            #Cria Bias da camada [0]
            biases.append(tf.get_variable("b0", [h_layers_dim[0]], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))
            #Cria Bias da camada [1]
            biases.append(tf.get_variable("b1", [h_layers_dim[1]], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))
            #Cria Bias da camada [2]
            biases.append(tf.get_variable("b2", [h_layers_dim[2]], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))
            #Cria Bias da camada [3]
            biases.append(tf.get_variable("b3", [self.INPUT_DIM], self.data_type, initializer=tf.contrib.layers.xavier_initializer()))

            #Calcula dados e adiciona ao tensorboard
            self.statistics.add_data(biases)

        #Array que armazena conexões
        connected_layers = []           
            
        with tf.variable_scope("Connect"):
            connected_layers.append(tf.matmul(input_states, weights[0]) + biases[0])
            connected_layers.append(tf.matmul(connected_layers[-1], weights[1]) + biases[1])
            connected_layers.append(tf.matmul(connected_layers[-1], weights[2]) + biases[2])
            connected_layers.append(tf.matmul(connected_layers[-1], weights[3]) + biases[3])

            #Calcula dados e adiciona ao tensorboard
            self.statistics.add_data(connected_layers)

        #Array que armazena ativações
        hidden_states = []           
            
        with tf.variable_scope("Activation"):
            for c_layer in connected_layers[:-1]:
                #Conecta o imput a camada 0 e aplica Relu
                hidden_states.append(tf.nn.relu(c_layer)) #Olhar o relu6, pode ser melhor
                #adiciona dados ao histogram do tensorboard
                tf.summary.histogram('hiddenStates', hidden_states[-1])

            #Conecta a camada 2 a camada 3 e gera output
            hidden_states.append(connected_layers[-1])
            #adiciona dados ao histogram do tensorboard
            tf.summary.histogram('hiddenStates', hidden_states[-1])

        
        with tf.variable_scope("Output"):
            #Output
            unnormalized_log_probs = hidden_states[-1]
            #Output com softmax, usado para determinar a ação
            action_probs = tf.nn.softmax(unnormalized_log_probs)

        with tf.variable_scope("Cross_entropy"):
            #Calcula cross entropy negativa para maximizar o reward
            neg_log_prob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=unnormalized_log_probs, labels=actions)
            #Calcula loss
            loss = tf.reduce_mean(neg_log_prob * rewards)  # reward guided loss
            #adiciona dados ao tensorboard            
            tf.summary.scalar('cross_entropy', loss)

        with tf.variable_scope("Optimize"):
            #Faz gradient descent e otimiza o modelo
            train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(loss)

        with tf.variable_scope("Eval"):
            eval_player = tf.placeholder(tf.int32, [1])
            tf.summary.scalar('Eval', eval_player[0])            
            
        return {
            "input_states": input_states,
            "actions": actions,
            "rewards": rewards,
            "eval": eval_player,
            "action_probs": action_probs,
            "train_step": train_step
        }
    
    #Calcula reward para cada action em um determinado estado
    def inference(self, input_state):
        #Usa rede para calcular quais actions são melhores em determinado estado
        return (self.sess.run(self.graph["action_probs"], feed_dict={self.graph["input_states"]: input_state})).squeeze()

    #Sorteia cenarios do historico de todos os (Estados, Ações, Rewards) para evitar overfiting por corelação temporal
    def sample_batch(self):
        #Cria array dos indices das jogadas selecionadas
        indexes = []
        #sorteia size jogadas distintas e captura os indices
        while(len(indexes) < self.batch_size):
            i = np.random.randint(0,len(self.replay_memory["states"]))
            if i not in indexes:
                indexes.append(i)
        #monta Lista de estados das jogadas selecionadas
        boards = [self.replay_memory["states"][i]for i in indexes]
        #monta Lista de ações das jogadas selecionadas
        actions = [self.replay_memory["actions"][i]for i in indexes]
        #monta Lista de rewards das jogadas selecionadas
        rewards = [self.replay_memory["rewards"][i]for i in indexes]
        
         #Retorna o batch de jogadas
        return np.array(boards).reshape((self.batch_size, self.INPUT_DIM)), np.array(actions), np.array(rewards)
    
    def add_samples_to_replay_mem(self):
        #Gera jogadas suficientes para compor o batch
        replay_memory_gen_count = 0
        while(replay_memory_gen_count <= self.batch_size):
            #Gera jogadas fazendo um jogo de rede X rede
            boards, moves, rewards = self.game.play(self)

            #Adiciona as jogadas ao replayMemory
            self.replay_memory["states"].extend(boards)
            self.replay_memory["actions"].extend(moves)
            self.replay_memory["rewards"].extend(rewards)
            
            #Update count
            replay_memory_gen_count += len(boards)

        #Remove itens excedentes da replayMemory se ela estiver maior que o limite
        if(len(self.replay_memory["states"]) >= self.replay_memory_limit):
            self.replay_memory["states"] = self.replay_memory["states"][len(boards)-1:]
            self.replay_memory["actions"] = self.replay_memory["actions"][len(moves)-1:]
            self.replay_memory["rewards"] = self.replay_memory["rewards"][len(rewards)-1:]
    
    def calculate_reward(self, player, actions):
        board_sum = np.sum(self.game.board.board)
        
        #Monta um array de shape de mesma dimenção das ações
        discounted_reward = np.zeros(np.array(actions).shape) 
        
        if(board_sum > 0):
            base_reward = 1*player
        elif(board_sum < 0):
            base_reward = -1*player
        else:
            base_reward = 0
            return discounted_reward

        #Captura ponto final para a ultima jogada do array
        discounted_reward[-1] = base_reward

        running_sum = 0
        #Propaga o reward aplicando o desconto para todas as jogadas tomadas
        for i in reversed(range(0, len(actions))):
            running_sum = running_sum * self.gamma + discounted_reward[i]
            discounted_reward[i] = running_sum
        # normaliza as rewards
        discounted_reward -= np.mean(discounted_reward)
        discounted_reward /= np.std(discounted_reward)
        
        #retorna rewards por ações tomadas
        return discounted_reward.tolist()
    
        #Testa rede contra um jogador aleatorio
    def eval_player(self):
        black_wins = 0
        for _ in range(self.eval_test_size):
            #Reinicia board ao estado inicial
            self.game.board.reset()

            #Printa jogo se verbose = true
            self.v or print(self.game.board)
            #Rede e jogador aleatorio tomam ações até q o jogo acabe
            while(not self.game.board.finished()):
                p1_move = self.game.player_move(self, self.game.BLACK)
                if(p1_move != -1):
                    self.v or print(self.game.board)
                if (self.game.random_move(self.game.WHITE) != -1):
                    self.v or print(self.game.board)
            if(self.game.board.score()[0] >= self.game.board.score()[1]):
                black_wins += 1 
        #retorna pontuação
        return black_wins
    
    def train(self):
        #Inicializa pesos e variaveis
        if(self.restore_file != ""):
            #restaura pesos e variaveis
            self.saver.restore(self.sess, self.restore_file)
        else:
            #Inicializa variaveis
            self.sess.run(tf.global_variables_initializer())

        self.statistics.start()
       
        #Add graph to tensorboard
        self.statistics.writer.add_graph(self.sess.graph)

        #Treina epochCount vezes
        for j in range(self.epoch_count):           
            #Adiciona states, actions e rewards ao replay_memory
            self.add_samples_to_replay_mem()

            #Recupera batchSize samples do replayMemory para o treinamento
            boards, actions, rewards = self.sample_batch()

            #Treina a rede
            _, summary = self.sess.run([
                                            self.graph["train_step"],
                                            self.statistics.merged_summary
                                        ], 
                                        feed_dict={
                                            self.graph["input_states"]: boards, 
                                            self.graph["actions"]: actions, 
                                            self.graph["rewards"]: rewards,
                                            self.graph["eval"]: [self.eval_player()]
                                        }
            )
            
            #Escreve dados para o tensorBoard
            self.statistics.writer.add_summary(summary)

            #A cada batchSize epochs, salva os pesos da rede
            if(j % self.batch_size == 0 and j != 0):
                #Salva pesos
                save_path = self.saver.save(self.sess, "./pesos/pesos-"+str(time.time())+".ckpt")
                print("Model saved in file: %s" % save_path)

                
        
                

brain = Brain(restore_file="")
brain.train()

